In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X_models,Y_labels,train_size=0.7,test_size = 0.30)

In [ ]:
random_forest_1 = RandomForestClassifier(max_features = 7,
                                            n_estimators = 100, max_depth=60)
random_forest_1.fit(X_train,Y_train)

In [ ]:
random_forest_2 = RandomForestClassifier(max_features = 7,
                                            n_estimators = 100,max_depth=70)
random_forest_2.fit(X_train,Y_train)

In [ ]:
random_forest_3 = RandomForestClassifier(max_features = 7,
                                            n_estimators = 100,max_depth=80)
random_forest_3.fit(X_train,Y_train)

In [ ]:
random_forest_4 = RandomForestClassifier(max_features = 7,
                                            n_estimators = 200,max_depth=60)
random_forest_4.fit(X_train,Y_train)

In [ ]:
random_forest_5 = RandomForestClassifier(max_features = 7,
                                            n_estimators = 200,max_depth=70)
random_forest_5.fit(X_train,Y_train)

In [ ]:
random_forest_6 = RandomForestClassifier(max_features = 7,
                                            n_estimators = 200,max_depth=80)
random_forest_6.fit(X_train,Y_train)

In [ ]:
random_forest_7 = RandomForestClassifier(max_features = 7,
                                            n_estimators = 300,max_depth=60)
random_forest_7.fit(X_train,Y_train)

In [ ]:
random_forest_8 = RandomForestClassifier(max_features = 7,
                                            n_estimators = 300,max_depth=70)
random_forest_8.fit(X_train,Y_train)

In [ ]:
random_forest_9 = RandomForestClassifier(max_features = 7,
                                            n_estimators = 300,max_depth=80)
random_forest_9.fit(X_train,Y_train)

In [ ]:
def fdr_RF(model,X_data,Y_data):
    Y_data = pd.DataFrame(Y_data)
    Y_data['Fraud Proba'] = model.predict_proba(X_data)[:,1].tolist()
    Y_data = Y_data.sort_values(by='Fraud Proba',ascending=False)
    total_bads = Y_data['fraud_label'][Y_data['fraud_label']==1].count()
    top_rows = int(len(X_data)*.03)
    sum_bads = Y_data['fraud_label'].head(top_rows)[Y_data['fraud_label']==1].count()
    fdr = sum_bads/total_bads
    return fdr*100

In [ ]:
time_results = pd.datetime.now()
results_dict_RF = {}
for i in range(1,10):
    curr_time = pd.datetime.now()
    model_name = "random_forest_"+str(i)
    results_dict_RF[model_name] = {'fdr':{}}
    
    # calculate fdr for training, testing, and validation sets
    results_dict_RF[model_name]['fdr']['train_fdr_30']=fdr_RF(vars()[model_name],X_train,Y_train)
    results_dict_RF[model_name]['fdr']['test_fdr_30']=fdr_RF(vars()[model_name],X_test,Y_test)
    results_dict_RF[model_name]['fdr']['oot_fdr_30']=fdr_RF(vars()[model_name],X_oot,Y_oot)
    
    print("Done with:",model_name, ";  time:",pd.datetime.now()-curr_time)
    
print("DONE!", pd.datetime.now()-time_results)

In [ ]:
results_dict_RF

In [ ]:
#Neural Work
import keras.backend as K
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [ ]:
#  function to compute FDR 
def get_FDR(y_scores, y_true):
    res_df = pd.DataFrame({'score':y_scores,'label': y_true}).sort_values(by='score',ascending=False)
    top3_res1 = res_df.head(round(y_true.shape[0]*0.03))
    top3_res2 = res_df.tail(round(y_true.shape[0]*0.03))
#     return np.maximum((top3_res1['label'].sum()/sum(y_true)),(top3_res1['label'].sum()/sum(y_true)))
    return (top3_res1['label'].sum()/sum(y_true))

In [ ]:

def recall_m(y_true, y_pred):
    y_true = K.ones_like(y_true) 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    all_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    
    recall = true_positives / (all_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    y_true = K.ones_like(y_true) 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_score(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
num_hidden = 30 # number of nodes in hidden layer
adam_par = Adam(learning_rate= 0.01, beta_1=0.9, beta_2=0.999, amsgrad=False) # adam algorithm parameters


model = Sequential()
model.add(Dense(num_hidden, input_dim=X_models.shape[1], activation='relu'))
# uncomment to add extra hidden layers
# model.add(Dense(20, input_dim=12, activation='relu'))
# model.add(Dense(15, input_dim=30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_score, precision_m, recall_m, tf.keras.metrics.AUC()])


In [ ]:
no_of_epochs = 10
batch = 1000 # assign batch size in training
class_weights = {0: 0.5, 1: 8.5} # assign class weight
start_time = pd.datetime.now()
history = model.fit(x=X_models, y=Y_labels, batch_size = batch,class_weight = class_weights, epochs = no_of_epochs, validation_split = 0.3)
print('duration: ', pd.datetime.now() - start_time)

In [ ]:
clas_pred_train = model.predict(X_train) # get class scores
clas_pred_test = model.predict(X_test) # get class scores
clas_pred_OOT =  model.predict(X_oot) # get class scores
print('Train FDR @ 3% is',get_FDR(np.squeeze(clas_pred_train),y_train))
print('Test FDR @ 3% is',get_FDR(np.squeeze(clas_pred_test),y_test))
print('OOT FDR is:',get_FDR(np.squeeze(clas_pred_OOT),Y_oot))

In [ ]:
num_hidden = 30 # number of nodes in hidden layer
adam_par = Adam(learning_rate= 0.01, beta_1=0.9, beta_2=0.999, amsgrad=False) # adam algorithm parameters


model = Sequential()
model.add(Dense(num_hidden, input_dim=X_train.shape[1], activation='relu'))
# uncomment to add extra hidden layers
# model.add(Dense(20, input_dim=12, activation='relu'))
# model.add(Dense(15, input_dim=30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_score, precision_m, recall_m, tf.keras.metrics.AUC()])
no_of_epochs = 20
batch = 1000 # assign batch size in training
class_weights = {0: 0.5, 1: 8.5} # assign class weight
start_time = pd.datetime.now()
history = model.fit(x=X_models, y=Y_labels, batch_size = batch,class_weight = class_weights, epochs = no_of_epochs, validation_split = 0.3)
print('duration: ', pd.datetime.now() - start_time)

In [ ]:
clas_pred_train = model.predict(X_train) # get class scores
clas_pred_test = model.predict(X_test) # get class scores
clas_pred_OOT =  model.predict(X_oot) # get class scores
print('Train FDR @ 3% is',get_FDR(np.squeeze(clas_pred_train),y_train))
print('Test FDR @ 3% is',get_FDR(np.squeeze(clas_pred_test),y_test))
print('OOT FDR is:',get_FDR(np.squeeze(clas_pred_OOT),Y_oot))

In [ ]:
num_hidden = 30 # number of nodes in hidden layer
adam_par = Adam(learning_rate= 0.01, beta_1=0.9, beta_2=0.999, amsgrad=False) # adam algorithm parameters


model = Sequential()
model.add(Dense(num_hidden, input_dim=X_train.shape[1], activation='relu'))
# uncomment to add extra hidden layers
# model.add(Dense(20, input_dim=12, activation='relu'))
# model.add(Dense(15, input_dim=30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_score, precision_m, recall_m, tf.keras.metrics.AUC()])
no_of_epochs = 30
batch = 1000 # assign batch size in training
class_weights = {0: 0.5, 1: 8.5} # assign class weight
start_time = pd.datetime.now()
history = model.fit(x=X_models, y=Y_labels, batch_size = batch,class_weight = class_weights, epochs = no_of_epochs, validation_split = 0.3)
print('duration: ', pd.datetime.now() - start_time)
clas_pred_train = model.predict(X_train) # get class scores
clas_pred_test = model.predict(X_test) # get class scores
clas_pred_OOT =  model.predict(X_oot) # get class scores
print('Train FDR @ 3% is',get_FDR(np.squeeze(clas_pred_train),y_train))
print('Test FDR @ 3% is',get_FDR(np.squeeze(clas_pred_test),y_test))
print('OOT FDR is:',get_FDR(np.squeeze(clas_pred_OOT),Y_oot))

In [ ]:
num_hidden = 10 # number of nodes in hidden layer
adam_par = Adam(learning_rate= 0.01, beta_1=0.9, beta_2=0.999, amsgrad=False) # adam algorithm parameters


model = Sequential()
model.add(Dense(num_hidden, input_dim=X_train.shape[1], activation='relu'))
# uncomment to add extra hidden layers
# model.add(Dense(20, input_dim=12, activation='relu'))
# model.add(Dense(15, input_dim=30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_score, precision_m, recall_m, tf.keras.metrics.AUC()])
no_of_epochs = 10
batch = 1000 # assign batch size in training
class_weights = {0: 0.5, 1: 8.5} # assign class weight
start_time = pd.datetime.now()
history = model.fit(x=X_models, y=Y_labels, batch_size = batch,class_weight = class_weights, epochs = no_of_epochs, validation_split = 0.3)
print('duration: ', pd.datetime.now() - start_time)
clas_pred_train = model.predict(X_train) # get class scores
clas_pred_test = model.predict(X_test) # get class scores
clas_pred_OOT =  model.predict(X_oot) # get class scores
print('Train FDR @ 3% is',get_FDR(np.squeeze(clas_pred_train),y_train))
print('Test FDR @ 3% is',get_FDR(np.squeeze(clas_pred_test),y_test))
print('OOT FDR is:',get_FDR(np.squeeze(clas_pred_OOT),Y_oot))

In [ ]:
num_hidden = 10 # number of nodes in hidden layer
adam_par = Adam(learning_rate= 0.01, beta_1=0.9, beta_2=0.999, amsgrad=False) # adam algorithm parameters


model = Sequential()
model.add(Dense(num_hidden, input_dim=X_train.shape[1], activation='relu'))
# uncomment to add extra hidden layers
# model.add(Dense(20, input_dim=12, activation='relu'))
# model.add(Dense(15, input_dim=30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_score, precision_m, recall_m, tf.keras.metrics.AUC()])
no_of_epochs = 20
batch = 1000 # assign batch size in training
class_weights = {0: 0.5, 1: 8.5} # assign class weight
start_time = pd.datetime.now()
history = model.fit(x=X_models, y=Y_labels, batch_size = batch,class_weight = class_weights, epochs = no_of_epochs, validation_split = 0.3)
print('duration: ', pd.datetime.now() - start_time)
clas_pred_train = model.predict(X_train) # get class scores
clas_pred_test = model.predict(X_test) # get class scores
clas_pred_OOT =  model.predict(X_oot) # get class scores
print('Train FDR @ 3% is',get_FDR(np.squeeze(clas_pred_train),y_train))
print('Test FDR @ 3% is',get_FDR(np.squeeze(clas_pred_test),y_test))
print('OOT FDR is:',get_FDR(np.squeeze(clas_pred_OOT),Y_oot))

In [ ]:
num_hidden = 10 # number of nodes in hidden layer
adam_par = Adam(learning_rate= 0.01, beta_1=0.9, beta_2=0.999, amsgrad=False) # adam algorithm parameters


model = Sequential()
model.add(Dense(num_hidden, input_dim=X_train.shape[1], activation='relu'))
# uncomment to add extra hidden layers
# model.add(Dense(20, input_dim=12, activation='relu'))
# model.add(Dense(15, input_dim=30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_score, precision_m, recall_m, tf.keras.metrics.AUC()])
no_of_epochs = 30
batch = 1000 # assign batch size in training
class_weights = {0: 0.5, 1: 8.5} # assign class weight
start_time = pd.datetime.now()
history = model.fit(x=X_models, y=Y_labels, batch_size = batch,class_weight = class_weights, epochs = no_of_epochs, validation_split = 0.3)
print('duration: ', pd.datetime.now() - start_time)
clas_pred_train = model.predict(X_train) # get class scores
clas_pred_test = model.predict(X_test) # get class scores
clas_pred_OOT =  model.predict(X_oot) # get class scores
print('Train FDR @ 3% is',get_FDR(np.squeeze(clas_pred_train),y_train))
print('Test FDR @ 3% is',get_FDR(np.squeeze(clas_pred_test),y_test))
print('OOT FDR is:',get_FDR(np.squeeze(clas_pred_OOT),Y_oot))

In [ ]:
import xgboost as xgb
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, KFold, StratifiedKFold, cross_val_score, cross_val_predict
X_train, X_test, y_train, y_test = train_test_split(X_models, Y_labels, test_size=0.3, train_size=0.7)

In [ ]:
# Determine the ratio between the negative and positive classes to use with the scale_pos_weight parameter in XGBoost
sum_pos = sum(y_train== 1.0)
print(sum_pos)
sum_neg = sum(y_train== 0.0)
print(sum_neg)
ratio = sum_neg / sum_pos
print(ratio)

In [ ]:
def fdr_XGB(data, topRows):
    topRows_fs = int(round(len(data)*topRows))
    data_topRows = data.head(topRows_fs)
    frauds_current = data_topRows.loc[:,'fraud_label']
    bads_all = data.loc[data['fraud_label'] == 1]
    FDR = sum(frauds_current) / len(bads_all)
    return FDR

In [ ]:
def make_data(model, X_data, y_data):
    fraud_proba = model.predict_proba(X_data)[:, 1]
    curr_data = X_data.copy()
    curr_data.insert(0, 'fraud_label', y_data)
    curr_data.insert(1, 'fraud_proba', fraud_proba)
    curr_data = curr_data.sort_values(['fraud_proba'], ascending=False)
    return curr_data

In [ ]:
start_xgbfit=pd.datetime.now()
xgb_1 =  xgb.XGBClassifier(objective='binary:logistic', learning_rate = 0.01, 
                           n_estimators=600, max_depth=5, scale_pos_weight=ratio)
xgb_1.fit(X_train, y_train)
print("DONE!", pd.datetime.now()-start_xgbfit)

In [ ]:
start_xgbfit=pd.datetime.now()
xgb_2 =  xgb.XGBClassifier(objective='binary:logistic', learning_rate = 0.01, 
                           n_estimators=800, max_depth=5, scale_pos_weight=ratio)
xgb_2.fit(X_train, y_train)
print("DONE!", pd.datetime.now()-start_xgbfit)

In [ ]:
start_xgbfit=pd.datetime.now()
xgb_3 =  xgb.XGBClassifier(objective='binary:logistic', learning_rate = 0.01, 
                           n_estimators=1000, max_depth=5, scale_pos_weight=ratio)
xgb_3.fit(X_train, y_train)
print("DONE!", pd.datetime.now()-start_xgbfit)

In [ ]:
start_xgbfit=pd.datetime.now()
xgb_4 =  xgb.XGBClassifier(objective='binary:logistic', learning_rate = 0.01, 
                           n_estimators=600, max_depth=4, scale_pos_weight=ratio)
xgb_4.fit(X_train, y_train)
print("DONE!", pd.datetime.now()-start_xgbfit)

In [ ]:
start_xgbfit=pd.datetime.now()
xgb_5 =  xgb.XGBClassifier(objective='binary:logistic', learning_rate = 0.01, 
                           n_estimators=800, max_depth=4, scale_pos_weight=ratio)
xgb_5.fit(X_train, y_train)
print("DONE!", pd.datetime.now()-start_xgbfit)

In [ ]:
start_xgbfit=pd.datetime.now()
xgb_6 =  xgb.XGBClassifier(objective='binary:logistic', learning_rate = 0.01, 
                           n_estimators=1000, max_depth=4, scale_pos_weight=ratio)
xgb_6.fit(X_train, y_train)
print("DONE!", pd.datetime.now()-start_xgbfit)

In [ ]:
start_xgbfit=pd.datetime.now()
xgb_7 =  xgb.XGBClassifier(objective='binary:logistic', learning_rate = 0.001, 
                           n_estimators=600, max_depth=5, scale_pos_weight=ratio)
xgb_7.fit(X_train, y_train)
print("DONE!", pd.datetime.now()-start_xgbfit)

In [ ]:
start_xgbfit=pd.datetime.now()
xgb_8 =  xgb.XGBClassifier(objective='binary:logistic', learning_rate = 0.001, 
                           n_estimators=800, max_depth=5, scale_pos_weight=ratio)
xgb_8.fit(X_train, y_train)
print("DONE!", pd.datetime.now()-start_xgbfit)

In [ ]:
start_xgbfit=pd.datetime.now()
xgb_9 =  xgb.XGBClassifier(objective='binary:logistic', learning_rate = 0.001, 
                           n_estimators=1000, max_depth=5, scale_pos_weight=ratio)
xgb_9.fit(X_train, y_train)
print("DONE!", pd.datetime.now()-start_xgbfit)

In [ ]:
start_xgbfit=pd.datetime.now()
xgb_10 =  xgb.XGBClassifier(objective='binary:logistic', learning_rate = 0.001, 
                           n_estimators=600, max_depth=4, scale_pos_weight=ratio)
xgb_10.fit(X_train, y_train)
print("DONE!", pd.datetime.now()-start_xgbfit)

In [ ]:
time_results=pd.datetime.now()

results_dict_XGB={}
for num in range(1,17):
    curr_time=pd.datetime.now()
    curr_model_name = "xgb_" + str(num)
    
    results_dict_XGB[curr_model_name]={'scores':{},
                                   'data':{},
                                   'FDR':{}
                                  }
    
    # Calculate the accuracy scores of the model 
    train_score = vars()[curr_model_name].score(X_train, y_train)
    test_score = vars()[curr_model_name].score(X_test, y_test)
    oot_score = vars()[curr_model_name].score(X_oot, Y_oot)
    
    # Save the accuracy scores of the model
    results_dict_XGB[curr_model_name]['scores']['train_score'] = train_score
    results_dict_XGB[curr_model_name]['scores']['test_score'] = test_score
    results_dict_XGB[curr_model_name]['scores']['oot_score'] = oot_score
    
    # Calculate the ".predict_proba" and make dataframes for all datasets
    train_data = make_data(vars()[curr_model_name], X_train, y_train) 
    test_data = make_data(vars()[curr_model_name], X_test, y_test) 
    oot_data = make_data(vars()[curr_model_name], X_oot, Y_oot) 
    
    # Save all the dataframes for the model
    results_dict_XGB[curr_model_name]['data']['train_data'] = train_data
    results_dict_XGB[curr_model_name]['data']['test_data'] = test_data
    results_dict_XGB[curr_model_name]['data']['oot_data'] = oot_data
    
    # Calculate the FDRs
    train_FDR = fdr_XGB(train_data, 0.03)
    test_FDR = fdr_XGB(test_data, 0.03)
    oot_FDR = fdr_XGB(oot_data, 0.03)
    
    # Save the FDRs
    results_dict_XGB[curr_model_name]['FDR']['train_FDR'] = train_FDR
    results_dict_XGB[curr_model_name]['FDR']['test_FDR'] = test_FDR
    results_dict_XGB[curr_model_name]['FDR']['oot_FDR'] = oot_FDR
    
    print("Done with:",curr_model_name, ";  time:",pd.datetime.now()-curr_time)

print("DONE!", pd.datetime.now()-time_results)

In [ ]:
for model in results_dict_XGB.keys():
    print(model)
    print(results_dict_XGB[model]['FDR'])
    print("")